### Imports

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("docker_viina/.env")

url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
ollama_base_url = os.getenv("OLLAMA_BASE_URL")
embedding_model_name = os.getenv("EMBEDDING_MODEL")
# Remapping for Langchain Neo4j integration
os.environ["NEO4J_URL"] = url

In [ ]:
%pip install -qU -r docker_viina/requirements.txt

In [ ]:
%pip install neo4j

In [21]:
import requests
from langchain_community.graphs import Neo4jGraph
# import streamlit as st
# from streamlit.logger import get_logger
# from chains import load_embedding_model
# from utils import create_constraints, create_vector_index
# from PIL import Image

# if Neo4j is local, you can go to http://localhost:7474/ to browse the database
neo4j_graph = Neo4jGraph(url=url, username=username, password=password)

create_actors_and_types = """MERGE (a_rus: Actor {name:'a_rus', description: 'Event initiated by Russian or Russian-aligned armed forces'});
    MERGE (a_ukr: Actor {name:'a_ukr', description: 'Event initiated by Ukrainian or Ukrainian-aligned armed forces'});
    MERGE (a_civ: Actor {name:'a_civ', description: 'Event initiated by civilians'});
    MERGE (a_other: Actor {name:'a_other', description: 'Event initiated by a third party (e.g. U.S., EU, Red Cross)'});
    MERGE (t_mil: Type {name:'t_mil', description: 'Event is about war/military operations'});
    MERGE (t_loc: Type {name:'t_loc', description: 'Event report includes reference to specific location'});
    MERGE (t_san: Type {name:'t_san', description: 'Event report mentions economic sanctions imposed on Russia'});
    MERGE (t_aad: Type {name:'t_aad', description: 'Anti-air defense, Buk, shoulder-fired missiles (Igla, Strela, Stinger)'});
    MERGE (t_airstrike: Type {name:'t_airstrike', description: 'Air strike, strategic bombing, helicopter strike'});
    MERGE (t_airalert: Type {name:'t_airalert', description: 'Air raid siren/alert'});
    MERGE (t_armor: Type {name:'t_armor', description: 'Tank battle or assault'});
    MERGE (t_arrest: Type {name:'t_arrest', description: 'Arrest by security services or detention of prisoners of war'});
    MERGE (t_artillery: Type {name:'t_artillery', description: 'Shelling by field artillery, howitzer, mortar, or rockets like Grad/BM-21, Uragan/BM-27, other Multiple Launch Rocket System (MRLS)'});
    MERGE (t_control: Type {name:'t_control', description: 'Establishment/claim of territorial control over population center'});
    MERGE (t_firefight: Type {name:'t_firefight', description: 'Any exchange of gunfire with handguns, semi-automatic rifles, automatic rifles, machine guns, rocket-propelled grenades (RPGs)'});
    MERGE (t_ied: Type {name:'t_ied', description: 'Improvised explosive device, roadside bomb, landmine, car bomb, explosion'});
    MERGE (t_raid: Type {name:'t_raid', description: 'Assault/attack by paratroopers or special forces, usually followed by a retreat'});
    MERGE (t_occupy: Type {name:'t_occupy', description: 'Occupation of territory or building'});
    MERGE (t_property: Type {name:'t_property', description: 'Destruction of property or infrastructure'});
    MERGE (t_cyber: Type {name:'t_cyber', description: 'Cyber operations, including DDOS attacks, website defacement'});
    MERGE (t_hospital: Type {name:'t_hospital', description: 'Attacks on hospitals and humanitarian convoys'});
    MERGE (t_milcas: Type {name:'t_milcas', description: 'Event report mentions military casualties'});
    MERGE (t_civcas: Type {name:'t_civcas', description: 'Event report mentions civilian casualties'});"""
for line in create_actors_and_types.split('\n'):
    neo4j_graph.query(line)
# neo4j_graph.query(create_actors_and_types)

# create_constraints
neo4j_graph.query(
    "CREATE CONSTRAINT actor_name IF NOT EXISTS FOR (a:Actor) REQUIRE (a.name) IS UNIQUE"
)
neo4j_graph.query(
    "CREATE CONSTRAINT type_name IF NOT EXISTS FOR (t:Type) REQUIRE (t.name) IS UNIQUE"
)
neo4j_graph.query(
    "CREATE CONSTRAINT event_id IF NOT EXISTS FOR (e:Event) REQUIRE (e.id_e) IS UNIQUE"
)

[]

In [ ]:
## test  csv
import csv

with open("docker_viina/import/test_1pd.csv", "r" ) as theFile:
    reader = csv.DictReader(theFile, delimiter=',')
    # i = 0
    for line in reader:
        # i+=1
        # if(i==2):
        #     print("...")
        #     break
            
        print(line)

In [17]:
import csv

csv_1pd = open("docker_viina/import/1pd.csv", "w", newline="")

with open("docker_viina/import/event_1pd_latest_2024.csv", "r" ) as theFile:
# with open("docker_viina/import/test_1pd.csv", "r" ) as theFile:
    reader = csv.DictReader(theFile, delimiter=',')
    
    fieldnames = ["event_id_1pd", "types", "actors"]
    writer = csv.DictWriter(csv_1pd, fieldnames=fieldnames, delimiter=',')
    writer.writeheader()
    
    # data = []
    a_headers :list[str] = []
    t_headers :list[str] = []
    i = 0
    for line in reader:
        if(i==0):
            for h in line.keys():
                if(h.startswith("a_")):
                    a_headers.append(h)
                if(h.startswith("t_")):
                    t_headers.append(h)
        # ---
        line["types"] = []
        for t in t_headers:
            if(line[t] == '1'):
                line["types"].append(t.removesuffix("_b"))
        # ---
        line["actors"] = []
        for a in a_headers:
            if(line[a] == '1'):
                line["actors"].append(a.removesuffix("_b"))
        # ---
        # data.append(line)
        writer.writerow({"event_id_1pd":line["event_id_1pd"], "types":line["types"], "actors":line["actors"]})
        # ---
        i+=1
        # ---
    # print({"data": data})
    # neo4j_graph.query(import_query, {"data": data})
csv_1pd.flush()
csv_1pd.close()

In [ ]:
# info.csv - merge instiejace Eventy po 'id_1pd' i set tytul itp

import_query = """
UNWIND $data AS row
//(CASE row.lang
//WHEN 'ua' THEN 'Ukrainian'
//WHEN 'ru' THEN 'Russian'
//ELSE 'Unknown' END) AS language
MERGE 
    (e: Event {
        id_1pd: row.event_id_1pd,
        id_e: row.event_ids[0],
        //date: row.date,
        //time: row.
        //language: language,
        //title: row.
        //url: row.
        //geonameid: row.geonameid
    })
FOREACH (type IN row.types |
    MERGE (e)-[:category]--(t:Type {name:type})
)
FOREACH (actor IN row.actors |
    MERGE (e)-[:category]--(a:Actor {name:actor})
)
"""